In [4]:
# Adding local Keras
import sys
import os
import numpy as np
from skimage import io
import itertools
from IPython.core.debugger import Tracer
%matplotlib inline
import matplotlib.pyplot as plt
KERAS_PATH = '/media/manish/Data/keras-master/'
sys.path.insert(0, KERAS_PATH)
sys.path.insert(0, os.path.join(KERAS_PATH, 'keras'))
sys.path.insert(0, os.path.join(KERAS_PATH, 'keras', 'layers'))
from keras.preprocessing.image import ImageDataGenerator

In [9]:
"""
data_weighted_loader consider the label for specific treatments
"""
def data_weighted_loader(path, batch_size, ignore_val=44, pos_val=255, neg_val=155, pos_class=3, neg_class=4):
    # pos_class and neg_class in the folder name for keras ImageDataGenerator input
    # 0,1,2 is for previous data. 3 and 4 is for new tcga data, which needs specific treatment

    def imerge(img_gen, mask_gen):
        
        for (img, img_labels), (mask, mask_labels) in itertools.izip(img_gen, mask_gen):
            Tracer()()
            
            # weight
            mask = np.expand_dims(mask[:,:,:,0], axis=3)
#             for img in mask:
#                 plt.figure()
#                 plt.imshow(img)
            print("Shape of the mask_labels: %s" % str(mask_labels))
            print("Shape of the label mask: %s" % str(mask.shape))
            
            weight = np.ones(mask.shape, np.float)
            weight[mask==ignore_val] = 0
            # In mask, ignored pixel has value ignore_val.
            # The weight of these pixel is set to zero, so they do not contribute to loss
            # The returned mask is still binary.

            # compute per sample
            for c, mask_label in enumerate(mask_labels):
                assert(mask_labels[c] == img_labels[c])
                mask_sample = mask[c]
                if mask_label == pos_class:
                    assert(np.where(mask_sample == neg_val)[0].size == 0)
                    mask_sample[mask_sample==pos_val] = 1
                elif mask_label == neg_class:
                    assert(np.where(mask_sample == pos_val)[0].size == 0)
                    mask_sample[mask_sample==neg_val] = 0
                else:
                	mask_sample /= 255

                mask_sample[mask_sample==ignore_val] = 0

            assert set(np.unique(mask)).issubset([0, 1])
            assert set(np.unique(weight)).issubset([0, 1])

            yield img/255.0, mask, weight, img_labels
            # yield norm_mean_img(img), mask, weight, img_labels

    train_data_gen_args = dict(
                    horizontal_flip=True,
                    vertical_flip=True,
                    zoom_range=0.1,
                    fill_mode='reflect')

    seed = 1234
    train_image_datagen = ImageDataGenerator(**train_data_gen_args).flow_from_directory(
                                path+'train/img',
                                class_mode="sparse",
                                target_size=(512, 512),
                                batch_size=batch_size,
                                seed=seed)
    train_mask_datagen = ImageDataGenerator(**train_data_gen_args).flow_from_directory(
                                path+'train/groundtruth',
                                class_mode="sparse",
                                target_size=(512, 512),
                                batch_size=batch_size,
                                seed=seed)

    test_image_datagen = ImageDataGenerator().flow_from_directory(
                                path+'val/img',
                                class_mode="sparse",
                                target_size=(512, 512),
                                batch_size=batch_size,
                                seed=seed)
    test_mask_datagen = ImageDataGenerator().flow_from_directory(
                                path+'val/groundtruth',
                                class_mode="sparse",
                                target_size=(512, 512),
                                batch_size=batch_size,
                                seed=seed)

    train_generator = imerge(train_image_datagen, train_mask_datagen)
    test_generator = imerge(test_image_datagen, test_mask_datagen)

    return train_generator, test_generator

training_dir = './data/BladderData/Segmentation/'
batch_size = 5
train_generator, _ = data_weighted_loader(training_dir, batch_size)
x_batch, y_batch, weight, _ = train_generator.next()h

Found 108755 images belonging to 5 classes.
Found 108755 images belonging to 5 classes.
Found 1600 images belonging to 3 classes.
Found 1600 images belonging to 3 classes.
> <ipython-input-9-4fbcd0c3968c>(14)imerge()
     13             # weight
---> 14             mask = np.expand_dims(mask[:,:,:,0], axis=3)
     15 #             for img in mask:

ipdb> plt.figure()
ipdb> plt.show(mask[0,:,:,1])


*** ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
ipdb> plt.show(np.squeeze(mask[0,:,:,:])
*** SyntaxError: unexpected EOF while parsing (<stdin>, line 1)
ipdb> plt.show(np.squeeze(mask[0,:,:,:]))


*** ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
ipdb> mask.any()
True
ipdb> plt.show(mask.any())


ipdb> mask.shape
(5, 512, 512, 3)
ipdb> mask[0,:,:,1]
array([[  44.,   44.,   44., ...,  155.,  155.,  155.],
       [  44.,   44.,   44., ...,  155.,  155.,  155.],
       [  44.,   44.,   44., ...,  155.,  155.,  155.],
       ..., 
       [  44.,   44.,   44., ...,  155.,  155.,  155.],
       [  44.,   44.,   44., ...,  155.,  155.,  155.],
       [  44.,   44.,   44., ...,  155.,  155.,  155.]], dtype=float32)
ipdb> mask_labels
array([4, 3, 4, 4, 3], dtype=int32)
ipdb> c
Shape of the mask_labels: [4 3 4 4 3]
Shape of the label mask: (5, 512, 512, 1)


In [ ]:
# Read the json file and whole slides and pick random points from annotation to crop the patch
# put positive patch in 1 folder and negative patch in 0 folder
